In [4]:
import pickle 
import pandas as pd
import spacy
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import transformers
from sentence_transformers import SentenceTransformer, util
import nltk
from nltk import tokenize
import pickle
import urllib.request as requests
import json
import statistics
import torch
from collections import defaultdict
import copy
import math
import os
import spacy

2022-08-21 23:31:31.345780: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
pathtosave=os.getcwd()

pathtosave=pathtosave + "/models/GPL/"

In [3]:
model = SentenceTransformer(pathtosave)

df=pd.read_csv('gmo_to_uniqueId.csv')
materialID=df['lastRunJob'].to_list()

with open('PreProcessed_segmented.pkl', 'rb') as f:
    newmetalist=pickle.load(f)

sampled_datadf=pd.read_csv("Sampled_Adword_Labeled.csv")

#For Normal process with target labels only

df1=pd.read_csv('ChildAdwordTargetDatabase.csv')    
target_dict=dict(zip(df1['Name'], df1.index))

In [4]:
def clean(text):
    text=text.lower()
    text=text.replace("(", "")
    text=text.replace(")", "")
    text=text.replace(";", "")
    text=text.replace(",", "")
    text=text.replace("+", "")
    text=text.replace(".", "")
    text=text.replace("&", "")
    return text

In [5]:
print(df1.head())

   Unnamed: 0                    Name  \
0           0              Automotive   
1           1    Books and Literature   
2           2    Business and Finance   
3           3               Education   
4           4  Events and Attractions   

                                                info  
0  Automotive Crossover Hatchback Microcar Off-Ro...  
1  Books and Literature Art and Photography Books...  
2  Business and Finance Business Business Account...  
3  Education College Education Primary Education ...  
4  Events and Attractions Fashion Events Malls & ...  


In [6]:
df1['info']=df1['info'].apply(clean)
targetlist=df1['Name'].to_list()
targetlistchild=df1['info']

In [7]:
print(targetlistchild)

0     automotive crossover hatchback microcar off-ro...
1     books and literature art and photography books...
2     business and finance business business account...
3     education college education primary education ...
4     events and attractions fashion events malls  s...
5     family and relationships dating parenting sing...
6     fine art costume dance design digital arts fin...
7     food  drink alcoholic beverages world cuisines...
8     healthy living fitness and exercise wellness c...
9     hobbies  interests musical instruments games a...
10    home  garden gardening remodeling  constructio...
11    medical health diseases and conditions allergi...
12    movies action and adventure movies romance mov...
13    music and audio adult contemporary music soft ...
14    pets birds cats dogs large animals reptiles fi...
15    pop culture celebrity deaths celebrity familie...
16    real estate apartments retail property hotel p...
17    religion  spirituality agnosticism spiritu

In [8]:
truthdatadf=pd.read_csv('AdwordLabeledDatabase.csv')
truthdatadf.head()

,Unnamed: 0.1,Unnamed: 0,materialId,lastRunJob,materialtitle,seriestitle,AssetCC,adword
0,3,3,GMO_00000000001313_01,FILE_MAF_20220211T205601Z_GMO_00000000001313_01,FULL HEARTS (EDITED),friday night lights,Is something happening\nbetween you and Riggin...,Sports
1,6,6,GMO_00000000001534_01,FILE_MAF_20220211T205607Z_GMO_00000000001534_01,MAY THE BEST MAN WIN (EDITED),friday night lights,BE HONEST WITH YOURSELF\nFOR TWO SECONDS AND T...,Sports
2,7,7,GMO_00000000001577_01,FILE_MAF_20210727T231031Z_GMO_00000000001577_01,GAVIN VOLURE,30 rock,"WE'RE AT A PARTY, LEMON,\nTRY TO LOOSEN UP A L...",Television
3,8,8,GMO_00000000001585_01,FILE_MAF_20210727T231030Z_GMO_00000000001585_01,DO-OVER,30 rock,"[bossa nova music] ♪ LLO, PUSSYCAT! (male voic...",Television
4,9,9,GMO_00000000001598_01,FILE_MAF_20210727T231031Z_GMO_00000000001598_01,CUTBACKS,30 rock,"[cheers] >> WHOO!\n>> WHOO! >> OKAY, GUYS, I J...",Television


In [9]:
def getadword(episode, truthdatadf):
    for i in range(len(truthdatadf.lastRunJob)):
        if episode == truthdatadf.lastRunJob[i]:
            return truthdatadf.adword[i]

In [10]:
sampled_datadf.head()

,Unnamed: 0.1,Unnamed: 0,materialId,lastRunJob,materialtitle,seriestitle,AssetCC,adword
0,665,665,GMO_00000000040741_01,FILE_MAF_20210731T020535Z_GMO_00000000040741_01,I KNEW YOU WHEN,friday night lights,"LKS, IS IS SLAMMIN' SAMMY MEADE,\nFO [radio]\n...",Sports
1,8244,8244,GMO_00000000370441_01,FILE_MAF_20220215T090159Z_GMO_00000000370441_01,Make Up or Break Up?,total bellas,>> NIKKI: Yeah! Welcome to Paris! Tonight on t...,Sports
2,678,678,GMO_00000000039091_01,FILE_MAF_20220213T121447Z_GMO_00000000039091_01,HOMECOMING (EDITED),friday night lights,"What's after\nhigh school, Brian? Me and Mack ...",Sports
3,777,777,GMO_00000000039094_01,FILE_MAF_20220213T173625Z_GMO_00000000039094_01,NEVERMIND (EDITED),friday night lights,"Here's to God in ten years from now, Street, g...",Sports
4,679,679,GMO_00000000039095_01,FILE_MAF_20220213T173619Z_GMO_00000000039095_01,WHAT TO DO WHILE YOU'RE WAITING (EDITED),friday night lights,"Dad? You're home! Hello, Brian. Waverly? Damn,...",Sports


In [11]:
listofepisodes=sampled_datadf['lastRunJob'].to_list()
len(listofepisodes)

380

In [ ]:
#listofepisodes=['FILE_MAF_20220211T205601Z_GMO_00000000001313_01', 'FILE_MAF_20220211T205607Z_GMO_00000000001534_01', 'FILE_MAF_20210727T231031Z_GMO_00000000001577_01' ]

In [12]:
def keydict(dataframe, targetlist):
    d=defaultdict(list)
    for target in targetlist:
        minidf=dataframe.loc[dataframe['adword']==target]
        minilist=minidf['lastRunJob'].to_list()
        d[target]=minilist
    
    return d



In [13]:
targetkey_filelist=keydict(sampled_datadf, targetlist)
print(targetkey_filelist)



defaultdict(<class 'list'>, {'Automotive': ['FILE_MAF_20211223T065128Z_GMO_00000000002584_01', 'FILE_MAF_20211223T064615Z_GMO_00000000106986_01', 'FILE_MAF_20211223T064819Z_GMO_00000000002709_01', 'FILE_MAF_20211222T231607Z_GMO_00000000294982_01', 'FILE_MAF_20211222T214634Z_GMO_00000000029298_01', 'FILE_MAF_20211222T214941Z_GMO_00000000121162_01', 'FILE_MAF_20211222T230805Z_GMO_00000000273733_01', 'FILE_MAF_20211223T065125Z_GMO_00000000027759_01', 'FILE_MAF_20211223T065130Z_GMO_00000000002708_01', 'FILE_MAF_20211223T063010Z_GMO_00000000002843_01', 'FILE_MAF_20211223T064314Z_GMO_00000000029163_01', 'FILE_MAF_20211222T214633Z_GMO_00000000027597_01', 'FILE_MAF_20211223T065127Z_GMO_00000000002637_01', 'FILE_MAF_20211223T065129Z_GMO_00000000009448_01', 'FILE_MAF_20211223T064919Z_GMO_00000000002707_01', 'FILE_MAF_20211222T214633Z_GMO_00000000027055_01', 'FILE_MAF_20211223T065126Z_GMO_00000000002685_01', 'FILE_MAF_20211223T065020Z_GMO_00000000009009_01', 'FILE_MAF_20211223T061005Z_GMO_0000000

In [ ]:
# #NRS bulk with top 1 save to file regular
# dict_to_save=defaultdict(list)
# for target in tqdm(targetlist):
#     scores=[]
#     for ep in tqdm(listofepisodes):
#         index=materialID.index(ep)
#         groundtruth=getadword(ep,truthdatadf)
#         # if groundtruth in target_dict:
#         #         x=target_dict[groundtruth]
#         x=target_dict[target]
#         #print(groundtruth)
#         print(x)
#         segmentscore=[]
#         for i in range(len(newmetalist[index])):
#             if len(newmetalist[index][i])==0:
#                 continue
#             emb1=model.encode(newmetalist[index][i]) #, convert_to_tensor=True)
#             emb2=model.encode(targetlist) #, convert_to_tensor=True)
#             cos_sim=util.cos_sim(emb1,emb2)

#             #For OG Class
#             MaxAggregation = {}
#             for i in range(cos_sim.shape[1]):
#                 all_sentence_combinations=[]
#                 for j in range(cos_sim.shape[0]):
#                     all_sentence_combinations.append([cos_sim[j][i], j, i])
#                 all_sentence_combinations = sorted(all_sentence_combinations, key=lambda x: x[0], reverse=True)
#                 MaxAggregation[i]=all_sentence_combinations[0]
                

#             segmentscore.append(MaxAggregation[x][0])

            
                    
#         avg=(sum(segmentscore)/len(segmentscore))
#         scores.append(avg)
        
#     print(scores)
#     dict_to_save[target]=scores

In [ ]:
# with open('NRS_top1_regular.pkl', 'wb') as f:
#     pickle.dump(dict_to_save, f)


In [ ]:
# with open('NRS_top1_regular.pkl', 'rb') as f:
#     NRS_dict=pickle.load(f)


In [23]:
def get_key(val):
    for key, value in target_dict.items():
         if val == value:
             return key
 
    return "key doesn't exist"

In [15]:
newmetalist[0][0]

['  just want to tell you that I appreciate you patch thing up with he .',
 'I would do anything for the woman that I love .',
 'they be go to come at we any way they can and when they do it be not go to be about heat sensor it be go to be about you .',
 'I know I be not the only guy in that company that know those sensor be faulty .',
 'I be sorry what be it you need again all the maintenance work order from the past six month .',
 'I can not let you have that without authorization .',
 'have you hear of the national transportation safety board evan that be your secretary unlawfully gather evidence which mean there will be no settlement of any kind .',
 'and tomorrow I be send this to the district attorney .',
 'your whole story about how louis get his name on the door be bullshit .',
 'jeff .',
 'it be over jessica .',
 'we be do .',
 'I can not believe you actually get they to admit what they do .',
 'you well believe it because it be happen and it would not be happen without you .'

In [16]:
targetlistchild[0]

'automotive crossover hatchback microcar off-road vehicles auto type budget cars certified pre-owned cars classic cars concept cars driverless cars performance cars car culture dash cam videos scooters auto buying and selling auto insurance auto recalls auto safety auto technology auto infotainment technologies auto navigation systems auto safety technologies auto body styles motorcycles road-side assistance auto parts auto repair auto shows auto rentals commercial trucks sedan station wagon suv van convertible coupe minivan pickup trucks green vehicles luxury cars individual brands'

In [5]:
#Relevance calculation per episode 
dictofrelevance=copy.deepcopy(targetkey_filelist)
for key, value in tqdm(targetkey_filelist.items()): #for each IAB Label
    if len(targetkey_filelist[key])==0:
        continue
    for i in tqdm(range(len(targetkey_filelist[key]))): #for each File
        index=materialID.index(targetkey_filelist[key][i])
        x=target_dict[key] #Index of the target label
        MaxAggregation=defaultdict(list)
        segmentscore=[]
        for l in range(len(newmetalist[index])): #For each Segment
            res=[]
            for j in range(len(targetlist)): 
            # print(len(newmetalist[index]))
                if len(newmetalist[index][l])==0: 
                    continue
                temp=" ".join(newmetalist[index][l])
                print(temp)
#                 emb1=model.encode(temp)
#                 emb2 = model.encode(targetlistchild[j])
#                 sim_score = util.cos_sim(emb1,emb2)
#                 MaxAggregation[j].append(sim_score)

        
#         for k,v in MaxAggregation.items():
#             temp=MaxAggregation[k]
#             avg=sum(temp)/len(temp)
#             MaxAggregation[k]=[avg]

            

#         dictofrelevance[key][i]={dictofrelevance[key][i]:MaxAggregation}
    

# with open('Relevance_gpl1.pkl', 'wb') as f:
#     pickle.dump(dictofrelevance, f)   
 





        #RS and NRS for OG Class

        # RSlist=MaxAggregation[x]
        # score=[]
        # for s in RSlist:
        #     score.append(s[0])

        # rs=sum(score)/len(score)

        # NRSlist=NRS_dict[key]
        # nrsdenom=sum(NRSlist)-rs/len(NRSlist)

        # NRS=rs/nrsdenom

        
        # #RS and ClassPredicted for Predicted Class

        # order=[]
        # for key in MaxAggregation.keys():
        #     l=MaxAggregation[key]
        #     summ=[]
        #     for val in l:
        #         summ.append(val[0])
        #     order.append(sum(summ))
        
        
        # max_val=max(order)
        # max_index=order.index(max_val)
        # ClassPredicted=get_key(max_index)

        # RSlistss=MaxAggregation[max_index]
        # scoress=[]
        # for s in RSlistss:
        #     scoress.append(s[0])

        # RS_CP=sum(scoress)/len(scoress)

        # d={targetkey_filelist[key][i]:{'NRS': NRS, "Class_Predicted": ClassPredicted, "RS_CP": RS_CP, "RS_OG": rs }}

        


        


        

    

NameError: name 'targetkey_filelist' is not defined

In [22]:
#Def Top1 Agg
def segmentcombiner(dic): #for segments to final dic of relevance scores
    dic1=copy.deepcopy(dic)
    filename=list(dic1.keys())[0]
    defdict=dic1[filename]
    for k,v in defdict.items():
        RSlist=defdict[k]
        score=[]
        for s in RSlist:
            score.append(s)

        rs=sum(score)/len(score)
        defdict[k]=rs
    d={filename:defdict}
    return d
    


In [ ]:
#Def Top3 and Harm Agg
# def segmentcombinertop3Harm(dic): #for segments to final dic of relevance scores
#     dic1=copy.deepcopy(dic)
#     filename=list(dic1.keys())[0]
#     defdict=dic1[filename]
#     for k,v in defdict.items():
#         RSlist=defdict[k][0]
#         score=[]
#         for s in RSlist:
#             score.append(s[0])

#         rs=sum(score)/len(score)
#         defdict[k]=rs
#     d={filename:defdict}
#     return d

In [16]:
with open('Relevance_gpl1.pkl', 'rb') as f:
     Relevance_dict=pickle.load(f)

In [18]:
print(NRS(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

...

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': 1.002466124966843, 'Books and Literature': 0.9964571619928626, 'Business and Finance': 1.0067585808814632, 'Education': 1.0044284919917874, 'Events and Attractions': 0.9933557029665432, 'Family and Relationships': 0.994045377330133, 'Food & Drink': 1.0121238074884409, 'Hobbies & Interests': 1.0022174213274457, 'Medical Health': 1.0092224134796304, 'Music and Audio': 1.0049593181821341, 'Pets': 1.0079436183987653, 'Real Estate': 1.0122665376019675, 'Religion & Spirituality': 0.9948947867280693, 'Science': 1.0033152294389254, 'Shopping': 1.0052494982599276, 'Sports': 0.9971569778835421, 'Style & Fashion': 1.0183665818478234, 'Technology & Computing': 1.0084620195803384, 'Television': 1.0009482829322638, 'Travel': 1.002830469229037}
None


In [17]:
#Def NRS for top1 
def NRS(dict):
    d={}
    for k,v in tqdm(dict.items()):#for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for target in tqdm(targetlist): #for each topic
            scoreforfiles=[]
            if len(dict[target]) == 0:
                continue
            for i in range(len(dict[k])): #for each file in label
                x=target_dict[target]
                combinedsegments=segmentcombiner(dict[k][i])
                key=list(combinedsegments.keys())[0]
                scoreforfiles.append(float(combinedsegments[key][x])) #combine segments 
            avg=sum(scoreforfiles)/len(scoreforfiles)
            d1[target]=avg
        d[k]=d1

    finaldict={}
    for k,v in d.items():
        numerator=d[k][k]
        denominator=[]
        for kk,vv in d.items():
            denominator.append(d[kk][k])
        avgdenominator=(sum(denominator)-numerator)/(len(denominator)-1)
        NRS=numerator/avgdenominator
        finaldict[k]=NRS
    print(finaldict)


            
        

In [69]:
print(RSMetrics(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 0.0}, 'Books and Literature': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Business and Finance': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Education': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Events and Attractions': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Family and Relationships': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Food & Drink': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Hobbies & Interests': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Medical Health': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Music and Audio': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Pets': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Real Estate': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Religion & Spirituality': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Science': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Shopping': {'RS15': 100.0, 'RS20': 100.0, 'RS35': 100.0}, 'Sports': {'RS15':

In [70]:
#Def RS at 0.15, 0.20, 0.35 and Mean RS 
def RSMetrics(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        scoreforfiles=[]
        for i in range(len(dict[k])): #for each file in label
            x=target_dict[k]
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            scoreforfiles.append(round(float(combinedsegments[key][x]),2))
        rs15=[]
        rs20=[]
        rs35=[]
        for item in scoreforfiles:
            if item >= 0.15:
                rs15.append(item)
            if item >= 0.20:
                rs20.append(item)
            if item >= 0.35:
                rs35.append(item)

        RS15=(len(rs15)/len(scoreforfiles))*100
        RS20=(len(rs20)/len(scoreforfiles))*100
        RS35=(len(rs35)/len(scoreforfiles))*100
        d1={"RS15":RS15, "RS20": RS20, "RS35": RS35}
        d[k]=d1
    
    return d
        
    


In [71]:
print(Prediction(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

{'Automotive': {'FILE_MAF_20211223T065128Z_GMO_00000000002584_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6017141643156709}, 'FILE_MAF_20211223T064615Z_GMO_00000000106986_01': {'ClassPredicted': 'Family and Relationships', 'RS_CP': 0.6163317611842932}, 'FILE_MAF_20211223T064819Z_GMO_00000000002709_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6056331133723035}, 'FILE_MAF_20211222T231607Z_GMO_00000000294982_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6088405691841746}, 'FILE_MAF_20211222T214634Z_GMO_00000000029298_01': {'ClassPredicted': 'Family and Relationships', 'RS_CP': 0.5976846186763813}, 'FILE_MAF_20211222T214941Z_GMO_00000000121162_01': {'ClassPredicted': 'Family and Relationships', 'RS_CP': 0.6017906276099642}, 'FILE_MAF_20211222T230805Z_GMO_00000000273733_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.6007698180894656}, 'FILE_MAF_20211223T065125Z_GMO_00000000027759_01': {'ClassPredicted': 'News and Politics', 'RS_CP': 0.61277304573440

In [19]:
#Def Predicted Class and RS of Class
def Prediction(dict):
    d={}
    for k,v in tqdm(dict.items()): #for each label
        if len(dict[k]) == 0:
            continue
        d1={}
        for i in range(len(dict[k])): #for each file in label
            combinedsegments=segmentcombiner(dict[k][i])
            key=list(combinedsegments.keys())[0]
            order=[]
            for kk in combinedsegments[key].keys():
                l=combinedsegments[key][kk]
                order.append(l)
            max_val=max(order)
            max_index=order.index(max_val)
            ClassPredicted=get_key(max_index)
            RS_CP=combinedsegments[key][max_index]
            d2={'ClassPredicted': ClassPredicted, 'RS_CP': RS_CP }
            d1[key]=d2
        d[k]=d1

    return d

            # 
            # scoreforfiles.append(float(combinedsegments[key][x]0))

            # order=[]
            # for key in MaxAggregation.keys():
            #     l=MaxAggregation[key]
            #     summ=[]
            #     for val in l:
            #         summ.append(val[0])
            #     order.append(sum(summ))
            
            
            # max_val=max(order)
            # max_index=order.index(max_val)
            # ClassPredicted=get_key(max_index)

            # RSlistss=MaxAggregation[max_index]
            # scoress=[]
            # for s in RSlistss:
            #     scoress.append(s[0])

            # RS_CP=sum(scoress)/len(scoress)
    

In [24]:
print(mAP(Relevance_dict))

  0%|          | 0/29 [00:00<?, ?it/s]

({'Automotive': 0.0, 'Books and Literature': 0.0, 'Business and Finance': 0.0, 'Education': 0.0, 'Events and Attractions': 0.0, 'Family and Relationships': 0.0, 'Food & Drink': 0.0, 'Hobbies & Interests': 100.0, 'Medical Health': 20.0, 'Music and Audio': 15.0, 'Pets': 0.0, 'Real Estate': 0.0, 'Religion & Spirituality': 0.0, 'Science': 0.0, 'Shopping': 0.0, 'Sports': 0.0, 'Style & Fashion': 35.0, 'Technology & Computing': 5.0, 'Television': 0.0, 'Travel': 0.0}, 8.75)


In [20]:
#Def mAP
def mAP(dic):
    res=Prediction(dic)
    d={}
    mAP=[]
    for k,v in res.items():
        classes=[]
        truth=[]

        for kk,vv in res[k].items():
            ClassPred=res[k][kk]['ClassPredicted']
            if ClassPred==k:
                classes.append(ClassPred)
            truth.append(k)
        precision=(len(classes)/len(truth))*100
        mAP.append(precision)
        d[k]=precision
    mAPfinal=sum(mAP)/len(mAP)
    return(d,mAPfinal)
    

